# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

# **Problem Description:**
Past studies in Sarcasm Detection mostly make use of Twitter datasets collected using hashtag based supervision but such datasets are noisy in terms of labels and language. Furthermore, many tweets are replies to other tweets and detecting sarcasm in these requires the availability of contextual tweets.
In this hands-on project, the goal is to build a model to detect whether a sentence is sarcastic or not, using Bidirectional LSTMs.

# Data Description:
News Headlines dataset for Sarcasm Detection. The dataset is collected from two news websites, theonion.com and huffingtonpost.com. This new dataset has the following advantages over the existing Twitter datasets:
Since news headlines are written by professionals in a formal manner, there are no spelling mistakes and informal usage. This reduces the sparsity and also increases the chance of finding pre-trained embedding.
Furthermore, since the sole purpose of TheOnion is to publish sarcastic news, we get high-quality labels with much less noise as compared to Twitter datasets.
Unlike tweets that reply to other tweets, the news headlines obtained are self-contained. This would help us in teasing apart the real sarcastic elements
Content
Each record consists of three attributes:
- is_sarcastic: 1 if the record is sarcastic otherwise 0
- headline: the headline of the news article
- article_link: link to the original news article. Useful in collecting supplementary data

## Install `Tensorflow2.0` 

In [ ]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 52kB/s 
     |████████████████████████████████| 450kB 35.7MB/s 
     |████████████████████████████████| 3.8MB 39.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=d6dd71dba6965d7971234d4c53dc96ef90c63778e956557bbc21f72d455b6bd7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


## Get Required Files from Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
#Set your project path 
project_path =  '/content/drive/My Drive/Colab Notebooks/Data/'

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [ ]:
import pandas as pd
headlines=pd.read_json('/content/drive/My Drive/Colab Notebooks/Data/Sarcasm_Headlines_Dataset.json',lines = True)

In [ ]:
headlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 626.1+ KB


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [ ]:
headlines.drop('article_link', axis=1, inplace=True)

In [ ]:
headlines.headline[0]

"former versace store clerk sues over secret 'black code' for minority shoppers"

## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [ ]:
headlines['word_count'] = [len(x.split()) for x in headlines['headline'].tolist()]
MAX_SEQ_LENGTH=headlines['word_count'].max()
MAX_SEQ_LENGTH

39

#**## Modelling**

## Import required modules required for modelling.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [ ]:
max_features = 10000
maxlen =MAX_SEQ_LENGTH
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_features,oov_token='OOV',filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
tokenizer.fit_on_texts(list(headlines['headline'].values))

Using TensorFlow backend.


# Define X and y for your model.

In [ ]:
X = tokenizer.texts_to_sequences(headlines['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(headlines['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0  308
    1  679 3337 2298   48  382 2576    1    6 2577 8434]
Number of Labels:  26709
0


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [ ]:
num_words=len(tokenizer.word_index)
num_words

29657

#**## Word Embedding**

## Get Glove Word Embeddings

In [ ]:
glove_file = project_path + "glove.6B.zip"

In [ ]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [ ]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

# Create a weight matrix for words in training docs

In [ ]:
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i-1] = embedding_vector
len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [ ]:
num_classes=2
model = Sequential()
model.add(Embedding(num_words,embedding_size,weights = [embedding_matrix]))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(units = 128, return_sequences = True,dropout=0.1, recurrent_dropout=0.2)))
model.add(GlobalMaxPool1D())
model.add(Dense(2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         5931400   
_________________________________________________________________
dropout (Dropout)            (None, None, 200)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         336896    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 6,268,813
Trainable params: 6,268,813
Non-trainable params: 0
______________________________________________

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [ ]:
batch_size = 100
epochs_count = 5
model.fit(X, y, batch_size=batch_size, epochs=epochs_count,validation_split=0.2, verbose=1, callbacks = [es])

Train on 21367 samples, validate on 5342 samples
Epoch 1/5
21367/21367 [==============================] - 101s 5ms/sample - loss: 0.5039 - accuracy: 0.7373 - val_loss: 0.3909 - val_accuracy: 0.8253
Epoch 2/5
21367/21367 [==============================] - 97s 5ms/sample - loss: 0.3158 - accuracy: 0.8614 - val_loss: 0.3322 - val_accuracy: 0.8540
Epoch 3/5
21367/21367 [==============================] - 98s 5ms/sample - loss: 0.2296 - accuracy: 0.9069 - val_loss: 0.3530 - val_accuracy: 0.8557
Epoch 4/5
21367/21367 [==============================] - 97s 5ms/sample - loss: 0.1763 - accuracy: 0.9295 - val_loss: 0.3562 - val_accuracy: 0.8538
Epoch 5/5
21367/21367 [==============================] - 97s 5ms/sample - loss: 0.1370 - accuracy: 0.9463 - val_loss: 0.4406 - val_accuracy: 0.8469


In [ ]:
print("Model accuracy is : " ,model.evaluate(X, y)[1]*100)

26709/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
!pip install -q wordcloud
import nltk

In [ ]:
#Lets submit our sample review on model and see outcome
from nltk import word_tokenize
nltk.download('punkt')
from keras.preprocessing import sequence
test=[]
for word in word_tokenize("grandpa almost uses streaming correctly in sentence"):
     test.append(tokenizer.word_index[word])
test=sequence.pad_sequences([test],maxlen=maxlen)
prediction = model.predict(test)
print(prediction)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[[0.99389356]]


In [ ]:
if (np.round(prediction) == 1):
    print("Review is sarcastic")
else:
    print("Review is not sarcastic")  

Review is sarcastic


# Conclusion:
Bidirectional LSTM with glove embeddings have given us good accuracy for sarcasm detection model. We can work further on model to make it more generic to evaluate sarcasm in sentences whihc are using words outside current vocabulary we used.